In [1]:
from collections import defaultdict

class Grafo:
    """
    Classe para representar um grafo orientado e verificar sua conectividade.
    """
    def __init__(self, vertices):
        """
        Inicializa o grafo com um número definido de vértices.
        """
        self.V = vertices
        self.grafo = defaultdict(list)

    def adiciona_aresta(self, u, v):
        """
        Adiciona uma aresta orientada do nó u para o nó v.
        """
        if u < self.V and v < self.V:
            self.grafo[u].append(v)
        else:
            print(f"Aviso: Vértice {u} ou {v} está fora do intervalo de 0 a {self.V-1}.")

    # --- Verificação de Conectividade Fraca ---

    def eh_fracamente_conexo(self):
        """
        Verifica se o grafo é fracamente conexo.
        Trata o grafo como não orientado e executa uma única busca.
        """
        # 1. Criar uma lista de adjacência não orientada
        adj_nao_orientado = defaultdict(list)
        for u in range(self.V):
            for v in self.grafo[u]:
                adj_nao_orientado[u].append(v)
                adj_nao_orientado[v].append(u)

        visitados = [False] * self.V

        # Encontra o primeiro vértice com arestas para iniciar a busca
        start_node = -1
        for i in range(self.V):
            if adj_nao_orientado[i]:
                start_node = i
                break

        # Se o grafo não tem arestas, mas tem mais de 1 nó, não é conexo.
        if start_node == -1:
            return self.V <= 1

        # 2. Executar DFS
        pilha = [start_node]
        visitados[start_node] = True
        count = 1

        while pilha:
            u = pilha.pop()
            for v in adj_nao_orientado[u]:
                if not visitados[v]:
                    visitados[v] = True
                    count += 1
                    pilha.append(v)

        # 3. Verificar se todos os nós com arestas foram visitados
        # Contamos quantos nós têm alguma conexão
        nodes_com_arestas = len(adj_nao_orientado.keys())
        return count == nodes_com_arestas

    # --- Verificação de Conectividade Forte (Algoritmo de Kosaraju) ---

    def _dfs_passo1(self, v, visitados, pilha):
        """DFS para preencher a pilha com os tempos de finalização."""
        visitados[v] = True
        for vizinho in self.grafo[v]:
            if not visitados[vizinho]:
                self._dfs_passo1(vizinho, visitados, pilha)
        pilha.append(v)

    def _obter_grafo_reverso(self):
        """Cria e retorna o grafo com todas as arestas invertidas."""
        g_rev = Grafo(self.V)
        for u in self.grafo:
            for v in self.grafo[u]:
                g_rev.adiciona_aresta(v, u)
        return g_rev

    def _dfs_passo2(self, v, visitados, g_rev):
        """DFS no grafo reverso para encontrar um Componente Fortemente Conexo."""
        visitados[v] = True
        for vizinho in g_rev.grafo[v]:
            if not visitados[vizinho]:
                self._dfs_passo2(vizinho, visitados, g_rev)

    def eh_fortemente_conexo(self):
        """
        Verifica se o grafo é fortemente conexo usando o algoritmo de Kosaraju.
        """
        # Grafo vazio ou com um nó é fortemente conexo
        if self.V == 0:
            return True

        # Passo 1: Preencher a pilha com base nos tempos de finalização
        pilha = []
        visitados = [False] * self.V
        for i in range(self.V):
            if not visitados[i]:
                self._dfs_passo1(i, visitados, pilha)

        # Passo 2: Obter o grafo reverso
        g_rev = self._obter_grafo_reverso()

        # Passo 3: Processar a pilha para encontrar SCCs
        visitados = [False] * self.V
        contador_scc = 0
        while pilha:
            v = pilha.pop()
            if not visitados[v]:
                self._dfs_passo2(v, visitados, g_rev)
                contador_scc += 1

        # Se houver apenas um SCC, o grafo é fortemente conexo
        return contador_scc == 1

# Exemplo de uso

In [11]:
import time

def test(graph, qtd=30):
    elapsed_fracamente_conexo = 0
    elapsed_fortemente_conexo = 0
    for i in range(qtd):
        start = time.perf_counter_ns()
        fracamente_conexo = graph.eh_fracamente_conexo()
        end = time.perf_counter_ns()
        elapsed_fracamente_conexo+= end - start

        start = time.perf_counter_ns()
        fortemente_conexo = graph.eh_fortemente_conexo()
        end = time.perf_counter_ns()
        elapsed_fortemente_conexo += end - start

    avg_fracamente_conexo = elapsed_fracamente_conexo / float(qtd)
    avg_fortemente_conexo = elapsed_fortemente_conexo / float(qtd)

    print(f"É fracamente conexo? {'Sim' if fracamente_conexo else 'Não'}")
    print(f"Número de testes: {qtd}")
    print(f"Tempo médio de verificação: {avg_fracamente_conexo:.2f} nanosegundos\n")
    print(f"É fortemente conexo? {'Sim' if fortemente_conexo else 'Não'}")
    print(f"Número de testes: {qtd}")
    print(f"Tempo médio de verificação: {avg_fortemente_conexo:.2f} nanosegundos")
    print("-" * 35)

In [12]:

if __name__ == "__main__":
    # Exemplo 1: Grafo Fortemente Conexo
    print("--- Grafo 1 (Fortemente Conexo) ---")
    g1 = Grafo(5)
    g1.adiciona_aresta(0, 1)
    g1.adiciona_aresta(1, 2)
    g1.adiciona_aresta(2, 3)
    g1.adiciona_aresta(3, 0)
    g1.adiciona_aresta(2, 4)
    g1.adiciona_aresta(4, 2)

    test(g1)
    # start = time.perf_counter_ns()
    # fracamente_conexo = g1.eh_fracamente_conexo()
    # end = time.perf_counter_ns()
    # elapsed_fracamente_conexo = end - start

    # start = time.perf_counter_ns()
    # fortemente_conexo = g1.eh_fortemente_conexo()
    # end = time.perf_counter_ns()
    # elapsed_fortemente_conexo = end - start

    # print(f"É fracamente conexo? {'Sim' if fracamente_conexo else 'Não'}")
    # print(f"Tempo de verificação: {elapsed_fracamente_conexo:.2f} nanosegundos\n")
    # print(f"É fortemente conexo? {'Sim' if fortemente_conexo else 'Não'}")
    # print(f"Tempo de verificação: {elapsed_fortemente_conexo:.2f} nanosegundos")
    # print("-" * 35)

    # Exemplo 2: Grafo Fracamente Conexo (mas não fortemente)
    print("--- Grafo 2 (Apenas Fracamente Conexo) ---")
    g2 = Grafo(4)
    g2.adiciona_aresta(0, 1)
    g2.adiciona_aresta(1, 2)
    g2.adiciona_aresta(2, 3)

    test(g2)
    # start = time.perf_counter_ns()
    # fracamente_conexo = g2.eh_fracamente_conexo()
    # end = time.perf_counter_ns()
    # elapsed_fracamente_conexo = end - start

    # start = time.perf_counter_ns()
    # fortemente_conexo = g2.eh_fortemente_conexo()
    # end = time.perf_counter_ns()
    # elapsed_fortemente_conexo = end - start

    # print(f"É fracamente conexo? {'Sim' if fracamente_conexo else 'Não'}")
    # print(f"Tempo de verificação: {elapsed_fracamente_conexo:.2f} nanosegundos\n")
    # print(f"É fortemente conexo? {'Sim' if fortemente_conexo else 'Não'}")
    # print(f"Tempo de verificação: {elapsed_fortemente_conexo:.2f} nanosegundos")
    # print("-" * 35)

    # Exemplo 3: Grafo Desconexo
    print("--- Grafo 3 (Desconexo) ---")
    g3 = Grafo(4)
    g3.adiciona_aresta(0, 1)
    g3.adiciona_aresta(2, 3)

    test(g1)
    # start = time.perf_counter_ns()
    # fracamente_conexo = g3.eh_fracamente_conexo()
    # end = time.perf_counter_ns()
    # elapsed_fracamente_conexo = end - start

    # start = time.perf_counter_ns()
    # fortemente_conexo = g3.eh_fortemente_conexo()
    # end = time.perf_counter_ns()
    # elapsed_fortemente_conexo = end - start

    # print(f"É fracamente conexo? {'Sim' if fracamente_conexo else 'Não'}")
    # print(f"Tempo de verificação: {elapsed_fracamente_conexo:.2f} nanosegundos\n")
    # print(f"É fortemente conexo? {'Sim' if fortemente_conexo else 'Não'}")
    # print(f"Tempo de verificação: {elapsed_fortemente_conexo:.2f} nanosegundos")
    # print("-" * 35)

--- Grafo 1 (Fortemente Conexo) ---
É fracamente conexo? Sim
Número de testes: 30
Tempo médio de verificação: 4526.67 nanosegundos

É fortemente conexo? Sim
Número de testes: 30
Tempo médio de verificação: 6453.33 nanosegundos
-----------------------------------
--- Grafo 2 (Apenas Fracamente Conexo) ---
É fracamente conexo? Sim
Número de testes: 30
Tempo médio de verificação: 5040.00 nanosegundos

É fortemente conexo? Não
Número de testes: 30
Tempo médio de verificação: 6586.67 nanosegundos
-----------------------------------
--- Grafo 3 (Desconexo) ---
É fracamente conexo? Sim
Número de testes: 30
Tempo médio de verificação: 3673.33 nanosegundos

É fortemente conexo? Sim
Número de testes: 30
Tempo médio de verificação: 5090.00 nanosegundos
-----------------------------------
